In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import concurrent

def getSoup(url):
    html_content = requests.get(url).text
    return BeautifulSoup(html_content, "lxml")

In [2]:
NATIONALITY = ['Nat.','Nationality','Nation']
DISCIPLINE = ['Discipline']
NAME = ['Player','Name']
AGE = ['Age','Date of birth','Pos']
YEAR = ['Year']
APPS = ['App']
GOALS = ['Goal']
TEAM = ['Team']
MANAGER = ['Manager']

In [3]:
def hasElementContaining(array,criteria):
    return len(list(filter(lambda x: criteria in x,array))) > 0

def hasElementContainingAnyOf(array,criterias):
    for criteria in criterias:
        if hasElementContaining(array,criteria):
            return True
    return False

def containsAnyOf(stringToCheck,values):
    for value in values:
        if value in stringToCheck:
            return True
    return False

def stripWiki(text):
    if text:
        return text.replace('/wiki/', '').strip()
    return None

def fullWiki(text):
    return "https://en.wikipedia.org" + text

def getSquadUrl(url):
    return 'https://en.wikipedia.org/wiki/2019-20_'+ url +'_season'

In [4]:
def getAllTables(url):
    soup = getSoup(url)
    tables = soup.find_all("table", attrs={"class": "wikitable"})
    title = soup.title.text.replace('- Wikipedia', '').strip()
    return title,tables

def getText(cell):
    return cell.text.replace('\n', '').strip()

def getHeaders(table):
    return list(map(lambda x:getText(x),table.find_all("th")))

def getIndexOf(table,values):
    headers = getHeaders(table)
    for i in range(len(headers)):
        if containsAnyOf(headers[i],values):
            return i
    return -1

def hasNoHeaders(row):
        return len(row.find_all("th")) == 0


def getRows(table):
    return table.tbody.find_all("tr")

def getBodyRows(table):
    rows = getRows(table)
    return list(filter(lambda x: hasNoHeaders(x),rows))

def extractLink(link):
    return link['href'].strip() if link else None

def getTextAndLink(cell):
    link = cell.find('a')
    url = extractLink(link)
    text = getText(cell)
    return text,url

def getAllLinks(cell):
    links = cell.find_all('a')
    return list(map(lambda x: (getText(x),extractLink(x)),links))
    
def getAllCells(row):
    cells = row.find_all("th") + row.find_all("td")
    return list(map(lambda x:getText(x),cells))

def getRowContaining(table,text):
    rows = getRows(table)
    for row in rows:
        cells = getAllCells(row)
        if hasElementContaining(cells,text):
            return getAllCells(row)
    return None
    

In [5]:
def isInternationalTable(table):
    headers = getHeaders(table)
    year = hasElementContainingAnyOf(headers,YEAR)
    apps = hasElementContainingAnyOf(headers,APPS)
    goals = hasElementContainingAnyOf(headers,GOALS)
    return year and apps and goals

def getAppearancesForPlayer(url):
    soup = getSoup(url)
    title,tables = getAllTables(url)
    internationalTables = list(filter(lambda x: isInternationalTable(x),tables))
    if len(internationalTables) > 0:
        internationalTable = internationalTables[0]
        totalRow = getRowContaining(internationalTable,'Total')
        appIndex = getIndexOf(internationalTable,APPS) -1 # Country in First Header
        if totalRow:
            return totalRow[appIndex]
    return 0

def isPlayerTable(table):
    headers = getHeaders(table)
    names = hasElementContainingAnyOf(headers,NAME)
    nationality = hasElementContainingAnyOf(headers,NATIONALITY) or hasElementContainingAnyOf(headers,DISCIPLINE)
    age = hasElementContainingAnyOf(headers,AGE)
    notTeam = not hasElementContainingAnyOf(headers,TEAM)
    return names and nationality and age and notTeam

def getDataFromPlayerTable(league,team,playerIndex,countryIndex,rows):
    players = []
    for row in rows:
        cells = row.find_all("td")
        url = None
        country = None
        url = None
        apps = None
        if(playerIndex == -1):
            print('ERROR: No Player Index - Stopping Early')
            return players
        if len(cells) < playerIndex:
            print('ERROR: Player Index Greater than Row Length - Stopping Early')
            return players
        links = getAllLinks(cells[playerIndex])
        (name, wikiPrefix) = getTextAndLink(cells[playerIndex])
        (tmp, countryWiki) = getTextAndLink(cells[countryIndex])
        if len(links) > 0:
            country = stripWiki(countryWiki)
            url = fullWiki(wikiPrefix)
            if countryIndex == -1:
#                 print('No Country Column for',name)
                emptyLinks = list(filter(lambda x: x[0]=='',links))
                if len(emptyLinks) > 0:
                    countryLink = emptyLinks[0]
                    country = stripWiki(countryLink[1])
                urlLinks = list(filter(lambda x: x[0]!='',links))
                if len(urlLinks) > 0:
                    urlLink = urlLinks[0]
                    url = fullWiki(urlLink[1])
        else:
            print('No links found for',name)
        result = (league,team,name,country,url)
#         print(result)
        players.append(result)
    return players

def getPlayerDataForTeam(league,url):
    title,tables = getAllTables(url)
    print('Starting ',title)
    playerTables = list(filter(lambda x: isPlayerTable(x),tables))
    if len(playerTables) > 0:
        playerTable = playerTables[0]
        playerIndex = getIndexOf(playerTable,NAME)
        countryIndex = getIndexOf(playerTable,NATIONALITY)
        playerRows = getBodyRows(playerTables[0])
        if len(playerRows)<17 and len(playerTables)>1:
            playerRows = playerRows + getBodyRows(playerTables[1])
        return getDataFromPlayerTable(league,title,playerIndex,countryIndex,playerRows)
    else:
        print('Messed Up Getting Players, ',url)
        return None


In [6]:
def getAppearances(url):
    soup = getSoup(url)
    title,tables = getAllTables(url)
    internationalTables = list(filter(lambda x: isInternationalTable(x),tables))
    if len(internationalTables) > 0:
        internationalTable = internationalTables[0]
        totalRow = getRowContaining(internationalTable,'Total')
        appIndex = getIndexOf(internationalTable,APPS) -1 # Country in First Header
        if totalRow:
            return totalRow[appIndex]
    return 0

def getPlayerDetails(league,team,name,country,url):
    print(name)
    return (league,team,name,country,getAppearances(url),url)

In [7]:
def getDetails(playerObject):
    league = playerObject[0]
    team = playerObject[1]
    name = playerObject[2]
    country = playerObject[3]
    url = playerObject[4]
    return getPlayerDetails(league,team,name,country,url)

# getDetails(('Prem','Arsenal','Leno','Germany','https://en.wikipedia.org/wiki/Bernd_Leno'))

In [8]:
# team = getPlayerDataForTeam('prem','https://en.wikipedia.org/wiki/2019%E2%80%9320_Manchester_United_F.C._season')
# results = []
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     futures = {executor.submit(getDetails,player) for player in team}
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             data = future.result()
#             results.append(data)
#         except Exception as exc:
#             print('%s generated an exception: %s' % (data[2],exc))
#         else:
#             print(data)
# print(results)

In [9]:
def writeTeamDataToFile(league,teamUrl,writer):
    team = getPlayerDataForTeam(league,teamUrl)
    if team:
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(getDetails,player) for player in team}
            for future in concurrent.futures.as_completed(futures):
                try:
                    data = future.result()
                    writer.writerow(data)
                except Exception as exc:
                    print('ERROR: Getting Details: %s' % (exc))
                else:
                    print(data)
    else:
        print('Error Getting Details for: ',teamUrl)

In [10]:
headers = ('League','Team','Player','Country','Appearances','URL')
def writePlayerDetailsToFile(leagueArray):
    with open(r'../data/everything.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        for team in leagueArray:
            league = team[0]
            teamUrl = team[1]
            writeTeamDataToFile(league,teamUrl,writer)

In [11]:
def isTeamsTable(table):
    headers = getHeaders(table)
    team = hasElementContainingAnyOf(headers,TEAM)
    return team 

def getDataFromTeamTable(table):
    rows = getBodyRows(table)
    teamIndex = getIndexOf(table,TEAM)
    teams = []
    dodgy = []
    for row in rows:
        cells = row.find_all("td")
        team = cells[teamIndex]
        (text,prefix) = getTextAndLink(team)
        if prefix:
            url = getSquadUrl(stripWiki(prefix))
            teams.append(url)
        else:
            dodgy.append(url)
    print('Dodgy Teams Were')
    print(dodgy)
    return teams

def getTeamsForLeague(url):
    title,tables = getAllTables(url)
    print(title)
    teamTables = list(filter(lambda x: isTeamsTable(x),tables))
    if len(teamTables) > 0:
        teamTable = teamTables[0]
        urls = getDataFromTeamTable(teamTable)
        return list(map(lambda x: [title,x],urls))
    else:
        print('Messed Up Getting Teams, ',url)
        

In [12]:
def writeTeamURLsToFile(urls,writer):
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(getTeamsForLeague,url) for url in urls}
        for future in concurrent.futures.as_completed(futures):
            try:
                data = future.result()
                for teamUrl in data:
                    writer.writerow(teamUrl)
            except Exception as exc:
                print('ERROR: %s' % (exc))
            else:
                print(data)

In [13]:
def getTeamsForLeagues(inputFile):
    with open(inputFile, newline='') as csvfile:
        leagues = list(csv.reader(csvfile))
    URLs = list(map(lambda x: x[1],leagues))
    print(URLs)
    with open(r'../data/teamList.csv', 'a') as f:
        writer = csv.writer(f)
        writeTeamURLsToFile(URLs,writer)
    
# getTeamsForLeagues('../data/leagues.csv')

In [14]:
def getTeamsFromFile():
    with open('../data/teamList.csv', newline='') as csvfile:
        teams = list(csv.reader(csvfile))
    return teams

In [16]:
teams = getTeamsFromFile()
writePlayerDetailsToFile(teams)

Starting  2019-20 Alianza Lima season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Alianza_Lima_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Alianza_Lima_season
Starting  2019-20 Alianza Universidad season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Alianza_Universidad_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Alianza_Universidad_season
Starting  2019-20 Ayacucho FC season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Ayacucho_FC_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Ayacucho_FC_season
Starting  2019-20 Escuela Municipal Deportivo Binacional season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Escuela_Municipal_Deportivo_Binacional_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Escuela_Municipal_Deportivo_Binacional_season
Starting  2019-20 Carlos A. Mannucci season
Messed Up Getting Pl

Starting  2019-20 Ashanti Gold S.C. season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Ashanti_Gold_S.C._season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Ashanti_Gold_S.C._season
Starting  2019-20 Berekum Chelsea F.C. season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Berekum_Chelsea_F.C._season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Berekum_Chelsea_F.C._season
Starting  2019-20 West African Football Academy season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_West_African_Football_Academy_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_West_African_Football_Academy_season
Starting  2019-20 Elmina Sharks F.C. season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Elmina_Sharks_F.C._season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Elmina_Sharks_F.C._season
Starting  2019-20 Legon Cities FC season
Messed Up G

Hamdou Elhouni('2019–20 Tunisian Ligue Professionnelle 1', '2019–20 Espérance Sportive de Tunis season', 'Mohamed Ali Yacoubi', 'Tunisia', 0, 'https://en.wikipedia.org/wiki/Mohamed_Ali_Yacoubi')

Youssef Mosrati('2019–20 Tunisian Ligue Professionnelle 1', '2019–20 Espérance Sportive de Tunis season', 'Amine Ben Hamida', 'Tunisia', 0, 'https://en.wikipedia.org/w/index.php?title=Amine_Ben_Hamida&action=edit&redlink=1')

Raouf Benguit
('2019–20 Tunisian Ligue Professionnelle 1', '2019–20 Espérance Sportive de Tunis season', 'Iheb Mbarki', 'Tunisia', 0, 'https://en.wikipedia.org/wiki/Iheb_Mbarki')
Mohamed Ali Ben Romdhane('2019–20 Tunisian Ligue Professionnelle 1', '2019–20 Espérance Sportive de Tunis season', 'Afif Jebali', 'Tunisia', 0, 'https://en.wikipedia.org/w/index.php?title=Afif_Jebali&action=edit&redlink=1')

Mohamed Amine Meskini('2019–20 Tunisian Ligue Professionnelle 1', '2019–20 Espérance Sportive de Tunis season', 'Abdelkader Bedrane', 'Algeria', 0, 'https://en.wikipedia.org/

('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Jerry Lalrinzuala', 'All_India_Football_Federation', 'Total', 'https://en.wikipedia.org/wiki/Jerry_Lalrinzuala')
('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Abhijit Sarkar', 'All_India_Football_Federation', 0, 'https://en.wikipedia.org/wiki/Abhijit_Sarkar_(footballer)')
('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Deepak Tangri', 'All_India_Football_Federation', 0, 'https://en.wikipedia.org/wiki/Deepak_Tangri')
('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Rahim Ali', 'All_India_Football_Federation', 0, 'https://en.wikipedia.org/wiki/Rahim_Ali')
('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Anirudh Thapa', 'All_India_Football_Federation', '24', 'https://en.wikipedia.org/wiki/Anirudh_Thapa')
('2019–20 Indian Super League season', '2019–20 Chennaiyin FC season', 'Germanpreet Singh', 'All_India_Football_Federatio

Liston Colaco('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Néstor Gordillo', 'Spain', 0, 'https://en.wikipedia.org/wiki/N%C3%A9stor_Gordillo')

('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Matthew Kilgallon', 'England', 0, 'https://en.wikipedia.org/wiki/Matthew_Kilgallon')
('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Hitesh Sharma', 'India', 0, 'https://en.wikipedia.org/wiki/Hitesh_Sharma_(footballer)')
('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Mohammad Yasir', 'India', 0, 'https://en.wikipedia.org/wiki/Mohammad_Yasir')
('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Marcelinho (C)', 'Brazil', 0, 'https://en.wikipedia.org/wiki/Marcelinho_Leite_Pereira')
('2019–20 Indian Super League season', '2019–20 Hyderabad FC season', 'Nikhil Poojari', 'India', 0, 'https://en.wikipedia.org/wiki/Nikhil_Poojari')
('2019–20 Indian Super League season', '2019–20 Hyderabad FC 

...

Gianluca Iannucci (Scholarship)('2019–20 A-League', '2019–20 Melbourne City FC season', 'Craig Noone', 'The_Football_Association', 0, 'https://en.wikipedia.org/wiki/Craig_Noone')

Yaya Dukuly (Scholarship)('2019–20 A-League', '2019–20 Melbourne City FC season', 'Jamie Maclaren', 'Football_Australia', '15', 'https://en.wikipedia.org/wiki/Jamie_Maclaren')

Richard Windbichler('2019–20 A-League', '2019–20 Melbourne City FC season', 'Lachlan Wales', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Lachlan_Wales')

Joe Gauci (Scholarship)
('2019–20 A-League', '2019–20 Melbourne City FC season', 'Adrián Luna', 'Uruguayan_Football_Association', 0, 'https://en.wikipedia.org/wiki/Adri%C3%A1n_Luna')
Stefan Colakovski (Scholarship)('2019–20 A-League', '2019–20 Melbourne City FC season', 'Connor Metcalfe', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Connor_Metcalfe')
Idrus Abdulahi (Scholarship)('2019–20 A-League', '2019–20 Melbourne City FC season', 'Moudi Najjar', 'Football_Au

Connor O'Toole
('2019–20 A-League', '2019–20 Newcastle Jets FC season', 'Lachlan Jackson', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Lachlan_Jackson')
Maki Petratos (Scholarship)('2019–20 A-League', '2019–20 Newcastle Jets FC season', 'Kosta Petratos', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Kosta_Petratos')

Angus Thurgate
('2019–20 A-League', '2019–20 Newcastle Jets FC season', 'John Koutroumbis', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/John_Koutroumbis')
Noah James (Scholarship)
('2019–20 A-League', '2019–20 Newcastle Jets FC season', 'Lewis Italiano', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Lewis_Italiano')
Nikolai Topor-Stanley (Vice-captain)
('2019–20 A-League', '2019–20 Newcastle Jets FC season', 'Bernie Ibini', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Bernie_Ibini-Isei')
('2019–20 A-League', '2019–20 Newcastle Jets FC season', "Roy O'Donovan", 'Football_Association_of_Ireland', 0, 'https://en.wikipedia

Luke Brattan('2019–20 A-League', '2019–20 Sydney FC season', 'Miloš Ninković', 'Football_Association_of_Serbia', '28', 'https://en.wikipedia.org/wiki/Milo%C5%A1_Ninkovi%C4%87')

('2019–20 A-League', '2019–20 Sydney FC season', 'Luke Ivanovic', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Luke_Ivanovic')
('2019–20 A-League', '2019–20 Sydney FC season', 'Anthony Cáceres', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Anthony_C%C3%A1ceres')
('2019–20 A-League', '2019–20 Sydney FC season', 'Chris Zuvela', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Chris_Zuvela')
('2019–20 A-League', '2019–20 Sydney FC season', 'Adam le Fondre', 'The_Football_Association', 0, 'https://en.wikipedia.org/wiki/Adam_le_Fondre')
('2019–20 A-League', '2019–20 Sydney FC season', 'Harry Van Der Saag (Scholarship)', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Harry_Van_Der_Saag')
('2019–20 A-League', '2019–20 Sydney FC season', 'Tom Heward-Belle', 'Football_Australia', 0,

Radosław Majewski (injured)('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', 'Fabian Monge (Scholarship)', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Fabian_Monge')

Tristan Prendergast (Injury replacement)('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', 'Mathieu Cordier', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Mathieu_Cordier')

('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', 'Daniel Wilmering (Scholarship)', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Daniel_Wilmering')
('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', 'Noah Pagden (Scholarship)', 'Football_Australia', 0, 'https://en.wikipedia.org/wiki/Noah_Pagden')
('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', 'Nicolai Müller (Injury replacement)', 'German_Football_Association', 0, 'https://en.wikipedia.org/wiki/Nicolai_M%C3%BCller')
('2019–20 A-League', '2019–20 Western Sydney Wanderers FC season', '

Starting  2019-20 Providence Park season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Providence_Park_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Providence_Park_season
Starting  2019-20 Rio Tinto Stadium season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Rio_Tinto_Stadium_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Rio_Tinto_Stadium_season
Starting  2019-20 Avaya Stadium season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Avaya_Stadium_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_Avaya_Stadium_season
Starting  2019-20 CenturyLink Field season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_CenturyLink_Field_season
Error Getting Details for:  https://en.wikipedia.org/wiki/2019-20_CenturyLink_Field_season
Starting  2019-20 Children's Mercy Park season
Messed Up Getting Players,  https://en.wikipedia.org/wiki/2019-20_Child